In [2]:
import cv2
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from past.utils import old_div
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique

2022-03-14 03:57:49.497279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 03:57:49.497307: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Base Functions

## Auxiliary function for Flip function

In [3]:
def boxesFromYOLO(imagePath,labelPath):
    '''
    Function to get an image and its corresponding bounding box
    
    Recieves:
    imagePath: complete path to the image file
    labelPath: complete path to the text file
    
    Returns:
    image: the image as a ndarray object
    boxes: the bounding boxes as a list of lists
    '''
    image = cv2.imread(imagePath)
    (hI, wI) = image.shape[:2]
    lines = [line.rstrip('\n') for line in open(labelPath)]
    boxes = []
    if lines != ['']:
        for line in lines:
            components = line.split(" ")
            category = components[0]
            x  = int(float(components[1])*wI - float(components[3])*wI/2)
            y = int(float(components[2])*hI - float(components[4])*hI/2)
            h = int(float(components[4])*hI)
            w = int(float(components[3])*wI)
            boxes.append((category, (x, y, w, h)))
    return (image,boxes)

In [4]:
def convert_labels(boxes,img_shape):
    '''
    Function to convert labels from the pixel scale to the 0 to 1 scale
    
    Recieves:
    boxes: a list of lists containing the bounding boxes from the boxesFromYOLO function
    img_shape: the image shape as a tuple or list
    
    Returns:
    norm_boxes: A list of lists of bounding boxes scaled form 0 to 1
    '''
    norm_boxes = []
    for line in boxes:
        h = round(int(line[1][3])/img_shape[0],6)
        w = round(int(line[1][2])/img_shape[1],6)
        y = round((int(line[1][1]) + h*img_shape[0]/2)/img_shape[0],6)
        x = round((int(line[1][0]) + w*img_shape[1]/2)/img_shape[1],6)
        norm_boxes.append([0,x,y,w,h])
    
    return norm_boxes
    

## Horizontal Flip

In [5]:
def horizontalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply horizontal flipping in an image and it's
    corresponding bounding box, if it has one. The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    # if the image has an associated label file, apply the clodsa function, else, apply the OpenCV flip 
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        hFlip = createTechnique("flip",{"flip":1})
        hFlipGenerator = transformer(hFlip)
        hFlipImg,hFlipBoxes = hFlipGenerator.transform(img,boxes)
        cv2.imwrite(path_to_save_imgs+'/aug_flipH_'+save_img, hFlipImg)
        norm_boxes = convert_labels(hFlipBoxes,hFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipH_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        hFlipImg = cv2.flip(img, 1)
        cv2.imwrite(path_to_save_imgs+'/aug_flipH_'+save_img, hFlipImg)
    
    return

## Vertical Flip

In [6]:
def verticalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply vertical flipping in an image and it's
    corresponding bounding box, if it has one. The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    # if the image has an associated label file, apply the clodsa function, else, apply the OpenCV flip 
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        vFlip = createTechnique("flip",{"flip":0})
        vFlipGenerator = transformer(vFlip)
        vFlipImg,vFlipBoxes = vFlipGenerator.transform(img,boxes)
        cv2.imwrite(path_to_save_imgs+'/aug_flipV_'+save_img, vFlipImg)
        norm_boxes = convert_labels(vFlipBoxes,vFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipV_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        vFlipImg = cv2.flip(img, 0)
        cv2.imwrite(path_to_save_imgs+'/aug_flipV_'+save_img, vFlipImg)
    
    return

## Double Flip

In [7]:
def doubleFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply vertical and horizontal flipping in an image and it's
    corresponding bounding box, if it has one. The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    # if the image has an associated label file, apply the clodsa function, else, apply the OpenCV flip 
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        dFlip = createTechnique("flip",{"flip":-1})
        dFlipGenerator = transformer(dFlip)
        dFlipImg,dFlipBoxes = dFlipGenerator.transform(img,boxes)
        cv2.imwrite(path_to_save_imgs+'/aug_flipD_'+save_img, dFlipImg)
        norm_boxes = convert_labels(dFlipBoxes,dFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipD_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        dFlipImg = cv2.flip(img, -1)
        cv2.imwrite(path_to_save_imgs+'/aug_flipD_'+save_img, dFlipImg)
    
    return

# Raise Hue

In [8]:
def raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply a raising of ^0.9 in the hue channel of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    
    img = cv2.imread(img_file)
    ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
    ##### begin
    imageHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    identityV = np.arange(256, dtype=np.dtype('uint8'))
    identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                          for i in np.arange(0, 256)]).astype("uint8")
    identityS = np.arange(256, dtype=np.dtype('uint8'))
    lut = np.dstack((identityH, identityS, identityV))

    # apply gamma correction using the lookup table
    imageHSV = cv2.LUT(imageHSV, lut)
    imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
    #### end
    
    cv2.imwrite(path_to_save_imgs+'/aug_hue_'+save_img, imageRGB)
    # if file has a label associated to it, it is coppied and renamed after the augmentation
    if in_labels == True:
        shutil.copy(path_labels+'/'+txt_file,path_to_save_labels)
        os.rename(path_to_save_labels+'/'+txt_file,path_to_save_labels+'/aug_hue_'+txt_file)

    return

# Constrast Adujstment

In [9]:
def adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply a contrast adjust to an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    
    #-----Reading the image-----------------------------------------------------
    img = cv2.imread(img_file)
    #-----Converting image to LAB Color model----------------------------------- 
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    cv2.imwrite(path_to_save_imgs+'/aug_contrst_'+save_img, final)
    # if file has a label associated to it, it is coppied and renamed after the augmentation
    if in_labels == True:
        shutil.copy(path_labels+'/'+txt_file,path_to_save_labels)
        os.rename(path_to_save_labels+'/'+txt_file,path_to_save_labels+'/aug_contrst_'+txt_file)

    return

# Combining functions

## Horizontal Flip and Raise Hue

In [10]:
def horizontalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply horizontal flip and raise the hue channel by 0.9 in an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        hFlip = createTechnique("flip",{"flip":1})
        hFlipGenerator = transformer(hFlip)
        hFlipImg,hFlipBoxes = hFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-hue_'+save_img, imageRGB)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(hFlipBoxes,hFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipH-hue_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        hFlipImg = cv2.flip(img, 1)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-hue_'+save_img, imageRGB)
    
    return

## Vertical Flip and Raise Hue

In [11]:
def verticalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply vertical flip and raise the hue channel by 0.9 in an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        vFlip = createTechnique("flip",{"flip":0})
        vFlipGenerator = transformer(vFlip)
        vFlipImg,vFlipBoxes = vFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-hue_'+save_img, imageRGB)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(vFlipBoxes,vFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipV-hue_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        vFlipImg = cv2.flip(img, 0)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-hue_'+save_img, imageRGB)
    
    return

## Double Flip and Raise Hue

In [12]:
def doubleFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply horizontal flip, vertical flip and raise the hue channel by 0.9 in an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        dFlip = createTechnique("flip",{"flip":-1})
        dFlipGenerator = transformer(dFlip)
        dFlipImg,dFlipBoxes = dFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-hue_'+save_img, imageRGB)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(dFlipBoxes,dFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipD-hue_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        dFlipImg = cv2.flip(img, -1)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))
        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-hue_'+save_img, imageRGB)
    
    return

## Horizontal Flip and Adjust Contrast

In [13]:
def horizontalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply horizontal flip and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        hFlip = createTechnique("flip",{"flip":1})
        hFlipGenerator = transformer(hFlip)
        hFlipImg,hFlipBoxes = hFlipGenerator.transform(img,boxes)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(hFlipBoxes,hFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipH-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        hFlipImg = cv2.flip(img, 1)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-contrst_'+save_img, final)
    
    return

## Vertical Flip and Adjust Contrast

In [14]:
def verticalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply vertical flip and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        vFlip = createTechnique("flip",{"flip":0})
        vFlipGenerator = transformer(vFlip)
        vFlipImg,vFlipBoxes = vFlipGenerator.transform(img,boxes)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(vFlipBoxes,vFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipV-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        vFlipImg = cv2.flip(img, 0)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-contrst_'+save_img, final)
    
    return

## Double Flip and Adjust Contrast

In [15]:
def doubleFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply horizontal flip, vertical flip and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        dFlip = createTechnique("flip",{"flip":-1})
        dFlipGenerator = transformer(dFlip)
        dFlipImg,dFlipBoxes = dFlipGenerator.transform(img,boxes)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(dFlipBoxes,dFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipD-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        dFlipImg = cv2.flip(img, -1)
        
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-contrst_'+save_img, final)
    
    return

## Raise Hue and Adjust Contrast

In [16]:
def raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to raise the hue channel by 0.9 and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    
    img = cv2.imread(img_file)
    ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
    ##### begin
    imageHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    identityV = np.arange(256, dtype=np.dtype('uint8'))
    identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                          for i in np.arange(0, 256)]).astype("uint8")
    identityS = np.arange(256, dtype=np.dtype('uint8'))
    lut = np.dstack((identityH, identityS, identityV))

    # apply gamma correction using the lookup table
    imageHSV = cv2.LUT(imageHSV, lut)
    imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
    #### end
        
    #-----Converting image to LAB Color model----------------------------------- 
    lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    cv2.imwrite(path_to_save_imgs+'/aug_hue-contrst_'+save_img, final)
    if in_labels == True:
        shutil.copy(path_labels+'/'+txt_file,path_to_save_labels)
        os.rename(path_to_save_labels+'/'+txt_file,path_to_save_labels+'/aug_hue-contrst_'+txt_file)

    return

## Horizontal Flip, Raise Hue and Adjust Contrast

In [17]:
def horizontalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply an horizontal flip, raise the hue channel by 0.9 and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        hFlip = createTechnique("flip",{"flip":1})
        hFlipGenerator = transformer(hFlip)
        hFlipImg,hFlipBoxes = hFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-hue-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(hFlipBoxes,hFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipH-hue-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        hFlipImg = cv2.flip(img, 1)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(hFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipH-hue-contrst_'+save_img, final)
    
    return

## Vertical Flip, Raise Hue and Adjust Contrast

In [18]:
def verticalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply an vertical flip, raise the hue channel by 0.9 and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        vFlip = createTechnique("flip",{"flip":0})
        vFlipGenerator = transformer(vFlip)
        vFlipImg,vFlipBoxes = vFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-hue-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(vFlipBoxes,vFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipV-hue-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        vFlipImg = cv2.flip(img, 0)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(vFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipV-hue-contrst_'+save_img, final)
    
    return

## Double Flip, Raise Hue and Adjust Contrast

In [19]:
def doubleFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels):
    '''
    Function to apply an horizontal flip, vertical flip, raise the hue channel by 0.9 and adjust the contrast of an image. 
    The function also writes the files
    
    Recieves:
    img_file: complete path to the image file
    txt_file: name of the text file containing the labels for the image file
    path_labels: path to te label files
    in_labels: boolean variable indicating if the image has or not a label file associated to it
    path_to_save_imgs: path were the modified image will be saved
    path_to_save_labels: path were the modified label file will be saved
    
    Returns:
    Nothing
    '''
    save_img = img_file.split('/')[-1]
    if in_labels == True:
        img,boxes = boxesFromYOLO(img_file,path_labels + '/' + txt_file)
        transformer = transformerGenerator("detection")
        dFlip = createTechnique("flip",{"flip":-1})
        dFlipGenerator = transformer(dFlip)
        dFlipImg,dFlipBoxes = dFlipGenerator.transform(img,boxes)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Salvar imagem no diretório de destino
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-hue-contrst_'+save_img, final)
        # Converter label para 0,1 e salvar no diretório destino
        norm_boxes = convert_labels(dFlipBoxes,dFlipImg.shape[:2])
        with open(path_to_save_labels+'/aug_flipD-hue-contrst_'+txt_file, "w") as label_file:
            for line in norm_boxes:
                line = [str(x) for x in line]
                line = ' '.join(line)
                line = line + '\n'
                label_file.write(line)
    else:
        img = cv2.imread(img_file)
        dFlipImg = cv2.flip(img, -1)
        
        ##### Code sample from https://github.com/joheras/CLoDSA/blob/master/clodsa/clodsa/techniques/raiseHueAugmentationTechnique.py
        ##### begin
        imageHSV = cv2.cvtColor(dFlipImg, cv2.COLOR_BGR2HSV)
        identityV = np.arange(256, dtype=np.dtype('uint8'))
        identityH = np.array([((old_div(i, 255.0)) ** 0.9) * 255
                              for i in np.arange(0, 256)]).astype("uint8")
        identityS = np.arange(256, dtype=np.dtype('uint8'))
        lut = np.dstack((identityH, identityS, identityV))

        # apply gamma correction using the lookup table
        imageHSV = cv2.LUT(imageHSV, lut)
        imageRGB = cv2.cvtColor(imageHSV, cv2.COLOR_HSV2BGR)
        #### end

        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(imageRGB, cv2.COLOR_BGR2LAB)
        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        cv2.imwrite(path_to_save_imgs+'/aug_flipD-hue-contrst_'+save_img, final)
    
    return

# Augmentation Loop

## Train

In [32]:
random.seed(0)

path_images = '/home/ld_brito/DeepL/dataset description/train/images'
path_labels = '/home/ld_brito/DeepL/dataset description/train/labels'
list_lables = os.listdir(path_labels)
path_to_save_imgs = '/home/ld_brito/DeepL/dataset description/train_augmented/images'
path_to_save_labels = '/home/ld_brito/DeepL/dataset description/train_augmented/labels'

p = 0.5 #probability that an image has to be aumgented

for img in os.listdir(path_images):

    img_file = path_images + '/' + img
    txt_file = img.split('.')[0]+'.txt'
    in_labels = txt_file in list_lables
    
    prob_hFlip = random.uniform(0, 1)
    if prob_hFlip <= p:
        horizontalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip = random.uniform(0, 1)
    if prob_vFlip <= p:
        verticalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_dFlip = random.uniform(0, 1)
    if prob_dFlip <= p:
        doubleFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_hue = random.uniform(0, 1)
    if prob_hue <= p:
        raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_contrst = random.uniform(0, 1)
    if prob_contrst <= p:
        adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hFlip_hue = random.uniform(0, 1)
    if prob_hFlip_hue <= p:
        horizontalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip_hue = random.uniform(0, 1)
    if prob_vFlip_hue <= p:
        verticalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_hue = random.uniform(0, 1)
    if prob_dFlip_hue <= p:
        doubleFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_hFlip_contrst = random.uniform(0, 1)
    if prob_hFlip_contrst <= p:
        horizontalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_vFlip_contrst = random.uniform(0, 1)
    if prob_vFlip_contrst <= p:
        verticalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_contrst = random.uniform(0, 1)
    if prob_dFlip_contrst <= p:
        doubleFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hue_contrst = random.uniform(0, 1)
    if prob_hue_contrst <= p:
        raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hFlip_hue_contrst = random.uniform(0, 1)
    if prob_hFlip_hue_contrst <= p:
        horizontalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip_hue_contrst = random.uniform(0, 1)
    if prob_vFlip_hue_contrst <= p:
        verticalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_hue_contrst = random.uniform(0, 1)
    if prob_dFlip_hue_contrst <= p:
        doubleFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    # copy original files to the augmented directory
    shutil.copy(img_file,path_to_save_imgs)
    if in_labels == True:
        shutil.copy(path_labels + '/' + txt_file,path_to_save_labels)

## Valid

In [33]:
random.seed(0)

path_images = '/home/ld_brito/DeepL/dataset description/valid/images'
path_labels = '/home/ld_brito/DeepL/dataset description/valid/labels'
list_lables = os.listdir(path_labels)
path_to_save_imgs = '/home/ld_brito/DeepL/dataset description/valid_augmented/images'
path_to_save_labels = '/home/ld_brito/DeepL/dataset description/valid_augmented/labels'

p = 0.5 #probability that an image has to be aumgented

for img in os.listdir(path_images):

    img_file = path_images + '/' + img
    txt_file = img.split('.')[0]+'.txt'
    in_labels = txt_file in list_lables
    
    prob_hFlip = random.uniform(0, 1)
    if prob_hFlip <= p:
        horizontalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip = random.uniform(0, 1)
    if prob_vFlip <= p:
        verticalFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_dFlip = random.uniform(0, 1)
    if prob_dFlip <= p:
        doubleFlip(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_hue = random.uniform(0, 1)
    if prob_hue <= p:
        raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_contrst = random.uniform(0, 1)
    if prob_contrst <= p:
        adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hFlip_hue = random.uniform(0, 1)
    if prob_hFlip_hue <= p:
        horizontalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip_hue = random.uniform(0, 1)
    if prob_vFlip_hue <= p:
        verticalFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_hue = random.uniform(0, 1)
    if prob_dFlip_hue <= p:
        doubleFlip_raiseHue(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_hFlip_contrst = random.uniform(0, 1)
    if prob_hFlip_contrst <= p:
        horizontalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_vFlip_contrst = random.uniform(0, 1)
    if prob_vFlip_contrst <= p:
        verticalFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_contrst = random.uniform(0, 1)
    if prob_dFlip_contrst <= p:
        doubleFlip_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hue_contrst = random.uniform(0, 1)
    if prob_hue_contrst <= p:
        raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_hFlip_hue_contrst = random.uniform(0, 1)
    if prob_hFlip_hue_contrst <= p:
        horizontalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    prob_vFlip_hue_contrst = random.uniform(0, 1)
    if prob_vFlip_hue_contrst <= p:
        verticalFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
        
    prob_dFlip_hue_contrst = random.uniform(0, 1)
    if prob_dFlip_hue_contrst <= p:
        doubleFlip_raiseHue_adjustContrast(img_file,txt_file,path_labels,in_labels,path_to_save_imgs,path_to_save_labels)
    
    # copy original files to the augmented directory
    shutil.copy(img_file,path_to_save_imgs)
    if in_labels == True:
        shutil.copy(path_labels + '/' + txt_file,path_to_save_labels)